# 빅데이터분석(21_2)
### 휴먼지능정보공학과 201810793 이연우
* 7주차 과제
---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#휴먼지능정보공학과-201810793-이연우" data-toc-modified-id="휴먼지능정보공학과-201810793-이연우-0.1">휴먼지능정보공학과 201810793 이연우</a></span></li></ul></li><li><span><a href="#7주:-zscore,-cdf-계산" data-toc-modified-id="7주:-zscore,-cdf-계산-1">7주: zscore, cdf 계산</a></span><ul class="toc-item"><li><span><a href="#1-1-성적데이터로-DataFrame을-생성." data-toc-modified-id="1-1-성적데이터로-DataFrame을-생성.-1.1">1-1 성적데이터로 DataFrame을 생성.</a></span></li><li><span><a href="#1-2-zscore-컬럼을-생성." data-toc-modified-id="1-2-zscore-컬럼을-생성.-1.2">1-2 zscore 컬럼을 생성.</a></span></li><li><span><a href="#1-3-cdf-컬럼을-생성." data-toc-modified-id="1-3-cdf-컬럼을-생성.-1.3">1-3 cdf 컬럼을 생성.</a></span></li></ul></li></ul></div>

## 7주: zscore, cdf 계산
성적데이터는 n이 적지만, 정규분포를 이룬다고 가정하자.

In [1]:
import os
import pyspark

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

### 1-1 성적데이터로 DataFrame을 생성.

In [2]:
from pyspark.sql.types import StructField, StructType, StringType, FloatType

marks = [
    ('김하나','English', 100.0),
    ('김하나','Math', 80.0),
    ('임하나','English', 70.0),
    ('임하나','Math', 100.0),
    ('김갑돌','English', 82.3),
    ('김갑돌','Math', 98.5)
]

marksRdd = spark.sparkContext.parallelize(marks)

marksSchema = StructType([
    StructField("NAME", StringType(), True),
    StructField("SUBJECT", StringType(), True),
    StructField("MARKS", FloatType(), True)
])

marksDf = spark.createDataFrame(marksRdd, marksSchema)

marksDf.printSchema()

marksDf.show()

root
 |-- NAME: string (nullable = true)
 |-- SUBJECT: string (nullable = true)
 |-- MARKS: float (nullable = true)

+------+-------+-----+
|  NAME|SUBJECT|MARKS|
+------+-------+-----+
|김하나|English|100.0|
|김하나|   Math| 80.0|
|임하나|English| 70.0|
|임하나|   Math|100.0|
|김갑돌|English| 82.3|
|김갑돌|   Math| 98.5|
+------+-------+-----+



### 1-2 zscore 컬럼을 생성.

zscore를 계산하려면, 평균과 표준편차를 알아야 한다.

계산식에 F함수를 직접 사용하면 오류가 발생한다. 따로 평균과 표준편차를 구해서 계산식에서 사용해야 한다.

In [4]:
import numpy as np

_marks=marksDf.rdd.map(lambda x:x[2]).collect()

xbar=np.mean(_marks)
sigmax=np.std(_marks)
sx=np.std(_marks,ddof=1)

marksDfZ=marksDf.withColumn('zscore',(marksDf['MARKS']-xbar)/sx)

marksDfZ.show()

+------+-------+-----+-------------------+
|  NAME|SUBJECT|MARKS|             zscore|
+------+-------+-----+-------------------+
|김하나|English|100.0|  0.902014804151829|
|김하나|   Math| 80.0| -0.662172786480269|
|임하나|English| 70.0| -1.444266581796318|
|임하나|   Math|100.0|  0.902014804151829|
|김갑돌|English| 82.3|-0.4822909748814927|
|김갑돌|   Math| 98.5| 0.7847007348544217|
+------+-------+-----+-------------------+



### 1-3 cdf 컬럼을 생성.

scipy.stats.norm.cdf() 함수는 데이터타입을 float로 맞추어 주어야 한다.

cdf는 평균=0, 표준편차=1을 기본 값으로 누적확률을 계산한다.

In [6]:
from scipy.stats import norm
import pyspark.sql.functions as F

_cdf=F.udf(lambda x: float(norm.cdf(x)))
cdf=marksDfZ.withColumn("cdf", _cdf(marksDfZ['zscore']))

cdf.show()

+------+-------+-----+-------------------+-------------------+
|  NAME|SUBJECT|MARKS|             zscore|                cdf|
+------+-------+-----+-------------------+-------------------+
|김하나|English|100.0|  0.902014804151829| 0.8164754981807292|
|김하나|   Math| 80.0| -0.662172786480269| 0.2539302463290559|
|임하나|English| 70.0| -1.444266581796318| 0.0743320011235712|
|임하나|   Math|100.0|  0.902014804151829| 0.8164754981807292|
|김갑돌|English| 82.3|-0.4822909748814927|0.31479962882028223|
|김갑돌|   Math| 98.5| 0.7847007348544217| 0.7836854740814176|
+------+-------+-----+-------------------+-------------------+

